In [ ]:
import requests
import datetime as dt
import pandas as pd
import numpy as np
import plotly.graph_objects as go


In [ ]:
URL = 'https://api.binance.com/api/v3/klines'
SYMBOL = 'BTCUSDT'
INTERVAL = '1m'
START_TIME = dt.datetime(2024, 1, 1, 8, 0, 0)
START_TIMESTAMP = int(START_TIME.timestamp()) * 1000

COLUMNS = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore']
SELECTED_COLUMNS = ['open_time', 'close']

params = {
    'symbol': SYMBOL,
    'interval': INTERVAL,
    'startTime': START_TIMESTAMP,
    'limit': 1000
}

response = requests.get(URL, params=params)
data = response.json()
df = pd.DataFrame(data, columns=COLUMNS)
df['open_time'] = pd.to_datetime(df['open_time'], unit='ms')
df['close'] = pd.to_numeric(df['close'])
df.head()


In [ ]:
selected_df = df[SELECTED_COLUMNS]
selected_df.head()


In [ ]:
returns = selected_df['close'].pct_change()
returns.head()


In [ ]:
terminal_return = selected_df['close'].iloc[-1] / selected_df['close'].iloc[0] - 1
terminal_return


In [ ]:
SHORT_WINDOW = 5
LONG_WINDOW = 25
SMA1 = "SMA-"+str(SHORT_WINDOW)
SMA2 = "SMA-"+str(LONG_WINDOW)

selected_df[SMA1] = selected_df['close'].rolling(SHORT_WINDOW).mean()
selected_df[SMA2] = selected_df['close'].rolling(LONG_WINDOW).mean()
selected_df.head()


In [ ]:
print(np.mean(selected_df['close'][:5]), selected_df['SMA-5'][4])
print(np.mean(selected_df['close'][1:6]), selected_df['SMA-5'][5])
print(np.mean(selected_df['close'][:25]), selected_df['SMA-25'][24])
print(np.mean(selected_df['close'][1:26]), selected_df['SMA-25'][25])


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=selected_df['open_time'], y=selected_df['close'], mode='lines', name='Price'))
fig.add_trace(go.Scatter(x=selected_df['open_time'], y=selected_df['SMA-5'], mode='lines', name='5m SMA'))
fig.add_trace(go.Scatter(x=selected_df['open_time'], y=selected_df['SMA-25'], mode='lines', name='25m SMA'))

fig.update_layout(
    title='Change in BTC/USDT Price and 5m, 25m Simple Moving Averages over time',
    xaxis_title='Time',
    yaxis_title='BTC/USDT Price',
)

fig.show()


In [ ]:
selected_df['SMA-5'] = selected_df['SMA-5'].shift(1)
selected_df['SMA-25'] = selected_df['SMA-25'].shift(1)

selected_df['signal'] = np.where(selected_df['SMA-5'] > selected_df['SMA-25'], 1, 0)
selected_df['signal'] = np.where(selected_df['SMA-5'] < selected_df['SMA-25'], -1, selected_df['signal'])
selected_df.dropna(inplace=True)
selected_df.head()


In [ ]:
selected_df['signal'].value_counts()


In [ ]:
selected_df['action'] = selected_df['signal'].diff()
selected_df['action'].value_counts()


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=selected_df['open_time'], y=selected_df['close'], mode='lines', name='Price'))
fig.add_trace(go.Scatter(x=selected_df['open_time'], y=selected_df['SMA-5'], mode='lines', name='5m SMA'))
fig.add_trace(go.Scatter(x=selected_df['open_time'], y=selected_df['SMA-25'], mode='lines', name='25m SMA'))

fig.add_trace(go.Scatter(
    x=selected_df['open_time'],
    y=selected_df['SMA-5'],
    mode='markers',
    marker=dict(
        size=10,
        color=selected_df['action'].apply(lambda val: 
            'green' if val == 2 else 'red' if val == -2 else 'rgba(0,0,0,0)'),
        symbol=selected_df['action'].apply(lambda val: 
            'triangle-up' if val == 2 else 'triangle-down' if val == -2 else 'circle'),
    ),
    name='Action'
))

fig.update_layout(
    title='Change in BTC/USDT Price and 5mins and 25mins Simple Moving Averages over time',
    xaxis_title='Time',
    yaxis_title='BTC/USDT Price',
)
fig.update_traces(showlegend=False, selector=dict(name='Action'))

fig.show()


In [ ]:
selected_df['log_return'] = np.log(selected_df['close']).diff() * selected_df['signal']
selected_df.head()


In [ ]:
print('Terminal return: {:.5%}'.format(np.exp(selected_df['log_return']).cumprod().iloc[-1] - 1))
